In [5]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 


In [14]:
# Step 1: Fetch S&P 500 data
def get_sp500_tickers():
    # Get S&P 500 tickers from Wikipedia
    sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(sp500_url, header=0)
    tickers = table[0]['Symbol'].tolist()
    return tickers

# Fetch S&P 500 tickers
tickers = get_sp500_tickers()
tickers = [ticker.replace('.', '-') for ticker in tickers]  # Adjust for yfinance

# Download historical data for the last year
data = yf.download(tickers, start="2023-01-01", end="2024-01-01")['Adj Close']

[*********************100%***********************]  503 of 503 completed

4 Failed downloads:
['SW', 'SOLV', 'GEV', 'AMTM']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2023-01-01 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1672549200, endDate = 1704085200")')


In [24]:
# Step 2: Preprocessing - Remove Excessive Missing Values
data_rem = data.dropna(thresh=int(0.8 * len(data.columns)), axis=0)  # At least 80% valid rows
data_rem = data_rem.dropna(thresh=int(0.8 * len(data)), axis=1)  # At least 80% valid columns


In [29]:
# Step 3: Calculate centered log returns
centered_log_returns = log_returns - log_returns.mean()

# Step 4: Compute covariance matrix of centered log returns
cov_matrix = centered_log_returns.cov()

# Step 5: Perform spectral decomposition
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

# Sort eigenvalues and eigenvectors
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]

# Dominant principal component
dominant_pc = eigenvectors[:, 0]

# Ensure positive orientation of the dominant PC
if dominant_pc[0] < 0:
    dominant_pc *= -1

# Create DataFrame for stock weights
stock_weights = pd.DataFrame({
    'Stock': centered_log_returns.columns,
    'Weight': dominant_pc
}).sort_values(by='Weight', ascending=False)

# Output
print("Top 10 stocks by weight in the dominant principal component:")
print(stock_weights.head(10))


Top 10 stocks by weight in the dominant principal component:
    Stock    Weight
263   KEY  0.110713
124   CZR  0.104890
372  PLTR  0.098506
161  ENPH  0.096758
357  PARA  0.096528
207  GNRC  0.093151
477   WBD  0.091262
81    CCL  0.090935
87    CFG  0.087999
71    BXP  0.087628
